<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

tf.__version__

'2.11.0'

In [24]:
vgg16 = tf.keras.applications.vgg16.VGG16(weights='imagenet')

In [25]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [28]:
vgg16_short = tf.keras.models.Model(vgg16.input, vgg16.layers[-5].output)

model = tf.keras.Sequential([
    vgg16_short, 
    tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'),
    # tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'),
    # tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu'),
    # tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'),
    # tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=4096, activation='relu'),
    tf.keras.layers.Dense(units=7, activation='softmax')
])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_12 (Functional)       (None, 7, 7, 512)         14714688  
                                                                 
 conv2d_35 (Conv2D)          (None, 7, 7, 1024)        4719616   
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 3, 3, 1024)       0         
 2D)                                                             
                                                                 
 conv2d_36 (Conv2D)          (None, 2, 2, 512)         4719104   
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 1, 1, 512)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 512)              